## Google Dance

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Starlight Dance Center'}, {'name': 'Arya Dance Academy'}, {'name': 'Wild Child Dance Studio'}, {'name': 'Class Act Performing Arts Studio'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'New Jersey Ballet Company and School'}, {'name': 'Village Dance Academy'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'Grooves Unlimited Dance Studio'}, {'name': 'Dance Palooza Studios'}, {'name': 'Kaye Lynn Dance Studio I'}, {'name': 'Marie Wildey School of Dance'}, {'name': 'Summit School of Dance'}, {'name': 'Cherney Dance Studio'}, {'name': 'Dance Academy of North Jersey'}, {'name': 'Lydia Johnson Dance School'}, {'name': 'The Kennedy Dancers'}, {'name': 'bboy bgirl lifestyle'}, {'name': 'Uptown Dance Studio'}, {'name': 'DanceWorks Studios'}, {'name': 'Performers Theatre Workshop'}, {'name': 'Just Dance!'}]


In [4]:
Data = []
geolocator = Nominatim()
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Dance Classes"
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]
                dataframe["city"] = orgAddress[2]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]
                dataframe["city"] = orgAddress[lenAddress-2]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]
                dataframe["city"] = orgAddress[lenAddress-1]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                print(orgAddress)
            try:
                locForLatLong = dataframe["address"] + dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=9ibkXbzsJKHb5gKFqZ-IBw&start=20&sa=N&ved=0ahUKEwi8v6GcqZXmAhWhrVkKHYXUB3EQ8tMDCPEC
2
start=20
start=40
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=9ibkXbzsJKHb5gKFqZ-IBw&start=40&sa=N&ved=0ahUKEwi8v6GcqZXmAhWhrVkKHYXUB3EQ8tMDCPEC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


20
Union+City+Music+%26+Dance+Academy,+118+40th+St,+Union+City,+NJ+07087
['Union City Music %26 Dance Academy', ' 118 40th St', ' Union City', ' NJ 07087']
 118 40th St Union City
118, 40th Street, Lincoln Harbor, Weehawken, Hudson County, New Jersey, 07087, United States of America
(40.7742930408163, -74.0213140816327)
The+Institute+of+Music+for+Children,+780+Salem+Ave,+Elizabeth,+NJ+07208
['The Institute of Music for Children', ' 780 Salem Ave', ' Elizabeth', ' NJ 07208']
 780 Salem Ave Elizabeth
780, Salem Avenue, Elizabeth, Union County, New Jersey, 07208, United States
(40.6813924411765, -74.2172814705882)
Just+Dancin',+187+Bloomfield+Ave,+Bloomfield,+NJ+07003
["Just Dancin'", ' 187 Bloomfield Ave', ' Bloomfield', ' NJ 07003']
 187 Bloomfield Ave Bloomfield
187, Bloomfield Avenue, Bloomfield, Staten Island, Richmond County, New York, 10314, United States of America
(40.6132314878049, -74.1833756829268)
Dance+Designs+Studio,+15-01+Pollitt+Dr,+Fair+Lawn,+NJ+07410
['Dance Designs Stu

29, Alden Street, Cranford, Union County, New Jersey, 07016, United States of America
(40.6567207142857, -74.3040674285714)
Dance+Xplosion,+601+Union+Blvd,+Totowa,+NJ+07512
['Dance Xplosion', ' 601 Union Blvd', ' Totowa', ' NJ 07512']
 601 Union Blvd Totowa
601, Union Boulevard, Totowa, Passaic County, New Jersey, 07512, United States
(40.893854, -74.225392)
Starlight+Dance+Center,+7+High+St,+Nutley,+NJ+07110
['Starlight Dance Center', ' 7 High St', ' Nutley', ' NJ 07110']
 7 High St Nutley
High Street, Nutley, Essex County, New Jersey, 07003, United States
(40.830143, -74.162407)
Wild+Child+Dance+Studio,+618+Ridge+Rd,+Lyndhurst,+NJ+07071
['Wild Child Dance Studio', ' 618 Ridge Rd', ' Lyndhurst', ' NJ 07071']
 618 Ridge Rd Lyndhurst
618, Ridge Road, North Arlington, Bergen County, New Jersey, 07071, United States of America
(40.8021511, -74.1231799)
Harmony+Dance+Theater,+1+Boonton+Ave,+Butler,+NJ+07405
['Harmony Dance Theater', ' 1 Boonton Ave', ' Butler', ' NJ 07405']
 1 Boonton Ave 

403, Broad Street, Bloomfield, Essex County, New Jersey, 07003, United States of America
(40.8070570098998, -74.192464954488)
8
start=80
start=100
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=9ibkXbzsJKHb5gKFqZ-IBw&start=100&sa=N&ved=0ahUKEwi8v6GcqZXmAhWhrVkKHYXUB3EQ8tMDCPEC
20
Shooting+Star+Dance+Center,+403+Broad+St,+Bloomfield,+NJ+07003
['Shooting Star Dance Center', ' 403 Broad St', ' Bloomfield', ' NJ 07003']
 403 Broad St Bloomfield
403, Broad Street, Bloomfield, Essex County, New Jersey, 07003, United States of America
(40.8070570098998, -74.192464954488)
Jill+Justin+Dance+Alliance,+1199+Amboy+Ave,+Edison,+NJ+08837
['Jill Justin Dance Alliance', ' 1199 Amboy Ave', ' Edison', ' NJ 08837']
 1199 Amboy Ave Edison
Amboy Avenue, Edison, Middlesex County, New Jersey, 08837:08840, United States
(40.535831, -74.342139)
Head+Over+Heels+Gymnastics,+Dance+%26+Cheer+Center,+287+Cheesequake+Rd,+Parlin,+NJ+08859
['Head Over Heels Gymna

Cant Find the address
Jersey+City+Ballroom,+189+Brunswick+St+3rd+FL,+Jersey+City,+NJ+07302
['Jersey City Ballroom', ' 189 Brunswick St 3rd FL', ' Jersey City', ' NJ 07302']
 189 Brunswick St 3rd FL Jersey City
Cant Find the address
Kaye-Lynn+Dance+Studio,+591+Cranbury+Rd,+East+Brunswick,+NJ+08816
['Kaye-Lynn Dance Studio', ' 591 Cranbury Rd', ' East Brunswick', ' NJ 08816']
 591 Cranbury Rd East Brunswick
591, Cranbury Road, Gillilandtown, East Brunswick Township, Middlesex County, New Jersey, 08816, United States of America
(40.4088022352941, -74.4209964705882)
Dance+Concepts+Dance+Studio,+101+Prosper+Way,+Brick+Township,+NJ+08723
['Dance Concepts Dance Studio', ' 101 Prosper Way', ' Brick Township', ' NJ 08723']
 101 Prosper Way Brick Township
Wepa Quenepa Puerto Rican Food, 101, Prosper Way, Brick Township, Ocean County, New Jersey, 08723, United States of America
(40.0632392, -74.145324)
Maywood+Dance+Center,+134+W+Pleasant+Ave,+Maywood,+NJ+07607
['Maywood Dance Center', ' 134 W Pl

13, Jefferson Avenue, Maplewood, Essex County, New Jersey, 07040, United States
(40.7347896, -74.270463)
Kavi's+School+of+Dance,+125+Washington+Valley+Rd,+Warren,+NJ+07059
["Kavi's School of Dance", ' 125 Washington Valley Rd', ' Warren', ' NJ 07059']
 125 Washington Valley Rd Warren
Washington Valley Road, Warren Township, Somerset County, New Jersey, 07059, United States of America
(40.6007136, -74.5389071)
Alma+Montuno+Latin+Dance,+LLC,+233+North+Ave,+Dunellen,+NJ+08812
['Alma Montuno Latin Dance', ' LLC', ' 233 North Ave', ' Dunellen', ' NJ 08812']
['', 'NJ', '08812']
 233 North Ave Dunellen
233, North Avenue, Dunellen, Middlesex County, New Jersey, 08812, United States
(40.5922170616927, -74.4623507215754)
14
start=140
start=160
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=9ibkXbzsJKHb5gKFqZ-IBw&start=160&sa=N&ved=0ahUKEwi8v6GcqZXmAhWhrVkKHYXUB3EQ8tMDCPEC
20
Alma+Montuno+Latin+Dance,+LLC,+233+North+Ave,+Dunellen,+NJ+08812
[

Morris Boulevard, Downtown Jersey City, Jersey City, Hudson County, New Jersey, 07302-4522, United States of America
(40.7146446, -74.0437455)
Today's+Dance+Center+Inc,+175+NJ-70+%23+2,+Medford,+NJ+08055
["Today's Dance Center Inc", ' 175 NJ-70 %23 2', ' Medford', ' NJ 08055']
 175 NJ-70 %23 2 Medford
Cant Find the address
Colts+Neck+Dance+%26+Performing,+410+NJ-34,+Colts+Neck,+NJ+07722
['Colts Neck Dance %26 Performing', ' 410 NJ-34', ' Colts Neck', ' NJ 07722']
 410 NJ-34 Colts Neck
JCT Route 34/537, Colts Neck Township, Monmouth County, New Jersey, 07722, United States of America
(40.3029295, -74.1779889)
Peak+Latin+Dancing,+734+Grand+Ave,+Ridgefield,+NJ+07657
['Peak Latin Dancing', ' 734 Grand Ave', ' Ridgefield', ' NJ 07657']
 734 Grand Ave Ridgefield
734, Grand Avenue, Ridgefield, Bergen County, New Jersey, 07657, United States
(40.840674, -74.005768)
Jersey+Shore+Dance+Academy,+555+Shrewsbury+Ave,+Shrewsbury,+NJ+07702
['Jersey Shore Dance Academy', ' 555 Shrewsbury Ave', ' Shrew

Cant Find the address
La+Creme+Modeling+%26+Acting,+1136+US+Highway+22+West,+Mountainside,+NJ+07092
['La Creme Modeling %26 Acting', ' 1136 US Highway 22 West', ' Mountainside', ' NJ 07092']
 1136 US Highway 22 West Mountainside
Cant Find the address
Perfect+Pointe+Performing+Arts,+30+Gail+Ct,+Sparta,+NJ+07871
['Perfect Pointe Performing Arts', ' 30 Gail Ct', ' Sparta', ' NJ 07871']
 30 Gail Ct Sparta
Cant Find the address
Dance+Corner+Inc,+335+Princeton+Hightstown+Rd,+West+Windsor+Township,+NJ+08550
['Dance Corner Inc', ' 335 Princeton Hightstown Rd', ' West Windsor Township', ' NJ 08550']
 335 Princeton Hightstown Rd West Windsor Township
335, Princeton Hightstown Road, West Windsor Township, Mercer County, New Jersey, 08512, United States of America
(40.2866978003687, -74.5657156296177)


In [5]:
#df1.to_excel("outputDance.xlsx") 
df1


,name,type,phoneNo,address,city,state,zipcode,location
0,Union City Music & Dance Academy,Dance Classes,,118 40th St,Union City,NJ,07087,"[40.7742930408163, -74.0213140816327]"
1,The Institute of Music for Children,Dance Classes,,780 Salem Ave,Elizabeth,NJ,07208,"[40.6813924411765, -74.2172814705882]"
2,Just Dancin',Dance Classes,,187 Bloomfield Ave,Bloomfield,NJ,07003,"[40.6132314878049, -74.1833756829268]"
3,Dance Designs Studio,Dance Classes,,15-01 Pollitt Dr,Fair Lawn,NJ,07410,"[40.9447716, -74.1260103]"
4,Salsa Fever On 2 Dance Academy,Dance Classes,,83 Franklin St,Jersey City,NJ,07307,"[40.7420044285714, -74.0483392857143]"
...,...,...,...,...,...,...,...,...
175,The Central NJ Ballet Theatre,Dance Classes,3,221 Broad St,Florence Township,NJ,08518,"[40.1207317753743, -74.8050583763107]"
176,Dance Dimensions By Michelle,Dance Classes,,270 S Sparta Ave,Sparta,NJ,07871,NaN
177,La Creme Modeling & Acting,Dance Classes,24,1136 US Highway 22 West,Mountainside,NJ,07092,NaN
178,Perfect Pointe Performing Arts,Dance Classes,,30 Gail Ct,Sparta,NJ,07871,NaN


## Google Music

In [6]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Rockness Music'}, {'name': 'Bill Wright School of Music'}, {'name': 'The Institute of Music for Children'}, {'name': 'Little Rockers - Cool Kids Music Classes'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Musical Munchkins of New Jersey'}, {'name': 'Rockness Music'}, {'name': 'Music Together of Bergen County'}, {'name': 'Music Together of Summit'}, {'name': 'Little Notes Music School'}, {'name': 'Edison School of Music and Dance'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'Joy Kids Music & Art (Joy Performing Art Center)'}, {'name': 'Music Together of Montclair'}, {'name': 'So i Heard Music Millburn'}, {'name': 'Rockness Music of Madison'}, {'name': 'Rhythm Kids'}, {'name': 'A Sound Start'}, {'name': 'ShreeRaga Carnatic Vocal Music School'}, {'name': 'Music Together With Kelly'}, {'name': 'The Extension Division at Cali School of Music'}]


In [7]:
Data = []
geolocator = Nominatim()
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Music Classes"
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]
                dataframe["city"] = orgAddress[2]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]
                dataframe["city"] = orgAddress[lenAddress-2]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]
                dataframe["city"] = orgAddress[lenAddress-1]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                print(orgAddress)
            try:
                locForLatLong = dataframe["address"] + dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)



1
/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=ZSfkXbiiEoe-gge36JOoDw&start=20&sa=N&ved=0ahUKEwj46IXRqZXmAhUHn-AKHTf0BPUQ8tMDCNsC
2
start=20
start=40
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=ZSfkXbiiEoe-gge36JOoDw&start=40&sa=N&ved=0ahUKEwj46IXRqZXmAhUHn-AKHTf0BPUQ8tMDCNsC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


20
Newark+School+of+the+Arts,+89+Lincoln+Park,+Newark,+NJ+07102
['Newark School of the Arts', ' 89 Lincoln Park', ' Newark', ' NJ 07102']
 89 Lincoln Park Newark
89, Lincoln Park, Teachers Village, Newark, Essex County, New Jersey, 07102, United States
(40.7256942631579, -74.1794780526316)
Kids'+Music+Round,+14+Gulick+Ln,+Plainsboro+Township,+NJ+08536
["Kids' Music Round", ' 14 Gulick Ln', ' Plainsboro Township', ' NJ 08536']
 14 Gulick Ln Plainsboro Township
14, Gulick Lane, Plainsboro Township, Middlesex County, New Jersey, 08536, United States of America
(40.3544950378446, -74.5807505891285)
Kidzmusik,+260+Grand+Ave,+Englewood,+NJ+07631
['Kidzmusik', ' 260 Grand Ave', ' Englewood', ' NJ 07631']
 260 Grand Ave Englewood
260, Grand Avenue, Englewood, Bergen County, New Jersey, 07631, United States of America
(40.8847180459598, -73.9777886310363)
Center+for+Music+and+Young+Children,+160+Lawrenceville+Pennington+Rd,+Lawrenceville,+NJ+08648
['Center for Music and Young Children', ' 160 L

505, NJ 18, Tanners Corner, East Brunswick Township, Middlesex County, New Jersey, 08816, United States
(40.4407724010877, -74.3943018714253)
School+of+Rock+Wayne,+1055+Hamburg+Turnpike+Unit+A-1,+Wayne,+NJ+07470
['School of Rock Wayne', ' 1055 Hamburg Turnpike Unit A-1', ' Wayne', ' NJ 07470']
 1055 Hamburg Turnpike Unit A-1 Wayne
Cant Find the address
Kids+Music+Together,+4910+43rd+Ave,+Woodside,+NY+11377
['Kids Music Together', ' 4910 43rd Ave', ' Woodside', ' NY 11377']
 4910 43rd Ave Woodside
43rd Avenue, Woodside, Queens, Queens County, New York, 11377, United States of America
(40.7420888, -73.892196)
Class+Act+Performing+Arts+Studio,+128+Irvington+Ave,+South+Orange,+NJ+07079
['Class Act Performing Arts Studio', ' 128 Irvington Ave', ' South Orange', ' NJ 07079']
 128 Irvington Ave South Orange
128, Irvington Avenue, Academy Heights, South Orange, Essex County, New Jersey, 07079, United States of America
(40.744295, -74.2564)
Nutley+School+of+Music,+213+Darling+Ave,+Nutley,+NJ+07

856, US 206, Hillsborough Township, Somerset County, New Jersey, 08844, United States of America
(40.4786684135514, -74.6551718240229)
8
start=80
start=100
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=ZSfkXbiiEoe-gge36JOoDw&start=100&sa=N&ved=0ahUKEwj46IXRqZXmAhUHn-AKHTf0BPUQ8tMDCNsC
20
New+Jersey+Academy+of+Music,+37+Stonehouse+Rd,+Basking+Ridge,+NJ+07920
['New Jersey Academy of Music', ' 37 Stonehouse Rd', ' Basking Ridge', ' NJ 07920']
 37 Stonehouse Rd Basking Ridge
Cant Find the address
Music+Together+of+Summit,+410+Bloomfield+Ave,+Montclair,+NJ+07042
['Music Together of Summit', ' 410 Bloomfield Ave', ' Montclair', ' NJ 07042']
 410 Bloomfield Ave Montclair
Bloomfield Avenue, Montclair, Essex County, New Jersey, 07042, United States
(40.8100079, -74.2129392)
Allegra+School+of+Music+%26+Arts,+856+US-206,+Hillsborough+Township,+NJ+08844
['Allegra School of Music %26 Arts', ' 856 US-206', ' Hillsborough Township', ' NJ 08844'

S. Broad Street, Trenton, Mercer County, New Jersey, 08611, United States
(40.2011346, -74.736033)
Raritan+Music+Center,+55+W+Somerset+St,+Raritan,+NJ+08869
['Raritan Music Center', ' 55 W Somerset St', ' Raritan', ' NJ 08869']
 55 W Somerset St Raritan
Raritan Music Center, 55, West Somerset Street, Raritan, Somerset County, New Jersey, 08869, United States of America
(40.5667708, -74.6354188)
Music+Together,+419+E+91st+St,+New+York,+NY+10128
['Music Together', ' 419 E 91st St', ' New York', ' NY 10128']
 419 E 91st St New York
419, East 91st Street, Yorkville, Manhattan Community Board 8, Manhattan, New York County, New York, 10128, United States of America
(40.77974935, -73.9457024370068)
New+Jersey+School+of+Music+-+Medford,+10+Union+St,+Medford,+NJ+08055
['New Jersey School of Music - Medford', ' 10 Union St', ' Medford', ' NJ 08055']
 10 Union St Medford
10, Union Street, Medford, Middlesex County, Massachusetts, 02155, United States
(42.4158031982465, -71.1097084225674)
John+J.+

Woodbury Glassboro Road, Buckingham Village, Gloucester County, New Jersey, 08080, United States of America
(39.7701419, -75.1349224)
14
start=140
start=160
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=ZSfkXbiiEoe-gge36JOoDw&start=160&sa=N&ved=0ahUKEwj46IXRqZXmAhUHn-AKHTf0BPUQ8tMDCNsC
20
ASPIRE+Studio,+394+Union+Blvd,+Totowa,+NJ+07512
['ASPIRE Studio', ' 394 Union Blvd', ' Totowa', ' NJ 07512']
 394 Union Blvd Totowa
394, Union Boulevard, Totowa, Passaic County, New Jersey, 07512, United States of America
(40.902455, -74.2141283333333)
Kindermusik+with+ms+k,+354+Maitland+Ave,+Teaneck,+NJ+07666
['Kindermusik with ms k', ' 354 Maitland Ave', ' Teaneck', ' NJ 07666']
 354 Maitland Ave Teaneck
354, Maitland Avenue, West Englewood, Teaneck Township, Bergen County, New Jersey, 07666, United States of America
(40.905254, -74.010320125)
Academy+of+Art,+727+Raritan+Ave,+Highland+Park,+NJ+08904
['Academy of Art', ' 727 Raritan Ave', ' Hig

233, Raritan Avenue, Highland Park, Middlesex County, New Jersey, 08904, United States of America
(40.498882131579, -74.4289780789474)
Cicely+L.+Tyson+Community+School+of+Performing+and+Fine+Arts,+35+Winans+St,+East+Orange,+NJ+07017
['Cicely L. Tyson Community School of Performing and Fine Arts', ' 35 Winans St', ' East Orange', ' NJ 07017']
 35 Winans St East Orange
Winans Street, East Orange, Essex County, New Jersey, 07017, United States
(40.7635127, -74.210541)
Music+%26+Arts,+240+N,+NJ-17,+Paramus,+NJ+07652
['Music %26 Arts', ' 240 N', ' NJ-17', ' Paramus', ' NJ 07652']
['', 'NJ', '07652']
 NJ-17 Paramus
NJ 17, Paramus, Bergen County, New Jersey, 07652, United States of America
(40.9506108, -74.0722792)
So+i+Heard+Music+Gillette,+977+Valley+Rd+unit+i,+Gillette,+NJ+07933
['So i Heard Music Gillette', ' 977 Valley Rd unit i', ' Gillette', ' NJ 07933']
 977 Valley Rd unit i Gillette
Cant Find the address
Center+for+Music+and+Young+Children,+144+Tices+Ln,+East+Brunswick,+NJ+08816
['Ce

In [8]:
#df.to_excel("outputMusic.xlsx") 
df2

,name,type,phoneNo,address,city,state,zipcode,location
0,Newark School of the Arts,Music Classes,,89 Lincoln Park,Newark,NJ,07102,"[40.7256942631579, -74.1794780526316]"
1,Kids' Music Round,Music Classes,6099369069,14 Gulick Ln,Plainsboro Township,NJ,08536,"[40.3544950378446, -74.5807505891285]"
2,Kidzmusik,Music Classes,8,260 Grand Ave,Englewood,NJ,07631,"[40.8847180459598, -73.9777886310363]"
3,Music Together of Bergen County,Music Classes,2016709993,NaN,NaN,NaN,NaN,NaN
4,Center for Music and Young Children,Music Classes,6099450308,160 Lawrenceville Pennington Rd,Lawrenceville,NJ,08648,"[40.3008898095238, -74.7479309047619]"
...,...,...,...,...,...,...,...,...
148,Cicely L. Tyson Community School of Performing...,Music Classes,9734148600,35 Winans St,East Orange,NJ,07017,"[40.7635127, -74.210541]"
149,Music & Arts,Music Classes,,NJ-17,Paramus,NJ,07652,"[40.9506108, -74.0722792]"
150,So i Heard Music Gillette,Music Classes,5,977 Valley Rd unit i,Gillette,NJ,07933,NaN
151,Center for Music and Young Children,Music Classes,6099450308,144 Tices Ln,East Brunswick,NJ,08816,"[40.4532230554597, -74.4151474535718]"


## Google Sports

In [9]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Life Time Athletic'}, {'name': 'Camp Pinebrook'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'The GYM For Kids'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'The Little Gym of Montclair'}, {'name': 'Karate For Youth Child Care'}]


In [10]:
Data = []
geolocator = Nominatim()
geolocator = Nominatim()
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Sports Classes"
        
        PhoneData = tr.find('span').text.split(' ')
        print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]
                dataframe["city"] = orgAddress[2]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]
                dataframe["city"] = orgAddress[lenAddress-2]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]
                dataframe["city"] = orgAddress[lenAddress-1]
                dataframe["state"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                print(orgAddress)
                    
            try:
                locForLatLong = dataframe["address"] + dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)



1
/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=xSfkXZ3RK8S45gLmgITIDw&start=20&sa=N&ved=0ahUKEwjdx4L_qZXmAhVEnFkKHWYAAfkQ8tMDCJ8C
2
start=20
start=40
https://www.google.com/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=xSfkXZ3RK8S45gLmgITIDw&start=40&sa=N&ved=0ahUKEwjdx4L_qZXmAhVEnFkKHWYAAfkQ8tMDCJ8C


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  
C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the defaul

20
['Fencing', 'school', '·', 'St.', "Paul's", 'Episcopal', 'ChurchChatham,', 'NJ(713)', '732-0766Open', '⋅', 'Closes', '6PM']
Integrity+Fencing+Studio,+200+Main+St,+Chatham,+NJ+07928
['Integrity Fencing Studio', ' 200 Main St', ' Chatham', ' NJ 07928']
 200 Main St Chatham
Saint Paul Episcopal Church, 200, Main Street, Chatham, Morris County, New Jersey, 07928, United States of America
(40.7402696, -74.3802555)
['Martial', 'arts', 'schoolJersey', 'City,', 'NJ(201)', '222-8996Closed', '⋅', 'Opens', '12PM', 'MonTheir', 'website', 'mentions', 'children', 'and', 'classes']
Family+Fitness+Karate+%26+Kickboxing+-+Central+Ave,+419+Central+Ave,+Jersey+City,+NJ+07307
['Family Fitness Karate %26 Kickboxing - Central Ave', ' 419 Central Ave', ' Jersey City', ' NJ 07307']
 419 Central Ave Jersey City
419, Central Avenue, Jersey City, Hudson County, New Jersey, 07307, United States of America
(40.7490070909091, -74.0474060909091)
['GymMaple', 'Shade', 'Township,', 'NJ(856)', '234-6445Closed', '⋅',

217, Clarksville Road, West Windsor Township, Mercer County, New Jersey, 08550, United States of America
(40.2982674559781, -74.6433897027504)
['Martial', 'arts', 'schoolDenville,', 'NJ(973)', '983-1008']
World+Class+Martial+Arts+Academy,+510+NJ-53,+Denville,+NJ+07834
['World Class Martial Arts Academy', ' 510 NJ-53', ' Denville', ' NJ 07834']
 510 NJ-53 Denville
510, East Main Street, Denville, Morris County, New Jersey, 07834, United States of America
(40.8752582973669, -74.4807745222845)
['PreschoolMontclair,', 'NJ(973)', '826-0350Closed', '⋅', 'Opens', '9AM', 'Mon']
Kidville,+516+Valley+Rd,+Montclair,+NJ+07043
['Kidville', ' 516 Valley Rd', ' Montclair', ' NJ 07043']
 516 Valley Rd Montclair
516, Valley Road, Montclair, Essex County, New Jersey, 07043, United States of America
(40.837221, -74.206941)
['Martial', 'arts', 'schoolCity', 'of', 'Orange,', 'NJ(973)', '674-5500Closed', '⋅', 'Opens', '3PM', 'Mon']
AJW+Martial+Arts+Academy+%26+Fitness+Center,+356+Main+St,+City+of+Orange,+NJ

1882, Springfield Avenue, Maplewood, Essex County, New Jersey, 07040, United States of America
(40.720718, -74.270127)
['Martial', 'arts', 'schoolBloomfield,', 'NJ(973)', '744-4934Closed', '⋅', 'Opens', '5PM', 'Mon']
Afro+Brazilian+Cultural+Center+of+New+Jersey,+554+Bloomfield+Ave,+Bloomfield,+NJ+07003
['Afro Brazilian Cultural Center of New Jersey', ' 554 Bloomfield Ave', ' Bloomfield', ' NJ 07003']
 554 Bloomfield Ave Bloomfield
554, Bloomfield Avenue, Bloomfield, Essex County, New Jersey, 07104, United States of America
(40.77275185, -74.18506265)
['Yoga', 'studioMaplewood,', 'NJ(973)', '763-2288Open', '⋅', 'Closes', '9PM']
Shakti+Yoga+%26+Living+Arts,+1861+Springfield+Ave+A,+Maplewood,+NJ+07040
['Shakti Yoga %26 Living Arts', ' 1861 Springfield Ave A', ' Maplewood', ' NJ 07040']
 1861 Springfield Ave A Maplewood
Springfield Avenue, Maplewood, Essex County, New Jersey, 07040, United States of America
(40.7229546, -74.2533239)
['Yoga', 'studioFair', 'Lawn,', 'NJ(201)', '203-0791Close

Cant Find the address
['GymSummit,', 'NJ(908)', '516-5040Open', '⋅', 'Closes', '7PM']
Equinox+Summit,+68+River+Rd,+Summit,+NJ+07901
['Equinox Summit', ' 68 River Rd', ' Summit', ' NJ 07901']
 68 River Rd Summit
68, River Road, Summit, Union County, New Jersey, 07901, United States
(40.7317625635542, -74.3742909475405)
['Boxing', 'gymWayne,', 'NJ(973)', '837-8220Closed', '⋅', 'Opens', '9AM', 'Mon"Kids', 'Boxing', 'Classes', 'on', 'Tuesdays', 'at', '5pm', 'and', 'Saturdays', 'at', '1230pm."']
TITLE+Boxing+Club+Wayne,+113+US-46,+Wayne,+NJ+07470
['TITLE Boxing Club Wayne', ' 113 US-46', ' Wayne', ' NJ 07470']
 113 US-46 Wayne
Tattoos are Forever NJ/NYC, 113, US 46, Mountain View, Wayne, Passaic County, New Jersey, 07470, United States
(40.8936131, -74.2606974)
['Dance', 'schoolParamus,', 'NJ(201)', '819-7989']
DDA+Dance+Academy,+40+Eisenhower+Dr+%23102,+Paramus,+NJ+07652
['DDA Dance Academy', ' 40 Eisenhower Dr %23102', ' Paramus', ' NJ 07652']
 40 Eisenhower Dr %23102 Paramus
40, Eisenhow

In [11]:
#df.to_excel("outputSports.xlsx") 
df3


,name,type,phoneNo,address,city,state,zipcode,location
0,Integrity Fencing Studio,Sports Classes,6,200 Main St,Chatham,NJ,07928,"[40.7402696, -74.3802555]"
1,Family Fitness Karate & Kickboxing - Central Ave,Sports Classes,,419 Central Ave,Jersey City,NJ,07307,"[40.7490070909091, -74.0474060909091]"
2,LittleSport,Sports Classes,,120 E Kings Hwy %23403,Maple Shade Township,NJ,08052,"[39.9332787373737, -74.9894579090909]"
3,Kid Fit NJ,Sports Classes,8,2040 Springdale Rd,Cherry Hill,NJ,08003,"[39.26813625, -84.53199775]"
4,Kids in Sports LLC,Sports Classes,830,1420 2nd Ave,New York,NY,10021,"[40.770143, -73.9571674]"
...,...,...,...,...,...,...,...,...
68,TITLE Boxing Club Wayne,Sports Classes,1230,113 US-46,Wayne,NJ,07470,"[40.8936131, -74.2606974]"
69,DDA Dance Academy,Sports Classes,2018197989,40 Eisenhower Dr %23102,Paramus,NJ,07652,"[40.9300192812892, -74.0778345166592]"
70,Grupo Liberdade de Capoeira,Sports Classes,615,2120 Springfield Ave,Vauxhall,NJ,07088,"[53.56396965, -2.4116526394168]"
71,Jewish Community Center MetroWest,Sports Classes,6,760 Northfield Ave,West Orange,NJ,07052,"[40.768356, -74.295825]"


## Merged all three dataframe into one DF

In [12]:

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

df = df.dropna(axis=0, subset=['location'])
df

,name,type,phoneNo,address,city,state,zipcode,location
0,Union City Music & Dance Academy,Dance Classes,,118 40th St,Union City,NJ,07087,"[40.7742930408163, -74.0213140816327]"
1,The Institute of Music for Children,Dance Classes,,780 Salem Ave,Elizabeth,NJ,07208,"[40.6813924411765, -74.2172814705882]"
2,Just Dancin',Dance Classes,,187 Bloomfield Ave,Bloomfield,NJ,07003,"[40.6132314878049, -74.1833756829268]"
3,Dance Designs Studio,Dance Classes,,15-01 Pollitt Dr,Fair Lawn,NJ,07410,"[40.9447716, -74.1260103]"
4,Salsa Fever On 2 Dance Academy,Dance Classes,,83 Franklin St,Jersey City,NJ,07307,"[40.7420044285714, -74.0483392857143]"
...,...,...,...,...,...,...,...,...
401,TITLE Boxing Club Wayne,Sports Classes,1230,113 US-46,Wayne,NJ,07470,"[40.8936131, -74.2606974]"
402,DDA Dance Academy,Sports Classes,2018197989,40 Eisenhower Dr %23102,Paramus,NJ,07652,"[40.9300192812892, -74.0778345166592]"
403,Grupo Liberdade de Capoeira,Sports Classes,615,2120 Springfield Ave,Vauxhall,NJ,07088,"[53.56396965, -2.4116526394168]"
404,Jewish Community Center MetroWest,Sports Classes,6,760 Northfield Ave,West Orange,NJ,07052,"[40.768356, -74.295825]"


In [13]:
df.to_excel("outputKidsActivities.xlsx") 

In [14]:
df=df.reset_index()
df.head()

,index,name,type,phoneNo,address,city,state,zipcode,location
0,0,Union City Music & Dance Academy,Dance Classes,,118 40th St,Union City,NJ,07087,"[40.7742930408163, -74.0213140816327]"
1,1,The Institute of Music for Children,Dance Classes,,780 Salem Ave,Elizabeth,NJ,07208,"[40.6813924411765, -74.2172814705882]"
2,2,Just Dancin',Dance Classes,,187 Bloomfield Ave,Bloomfield,NJ,07003,"[40.6132314878049, -74.1833756829268]"
3,3,Dance Designs Studio,Dance Classes,,15-01 Pollitt Dr,Fair Lawn,NJ,07410,"[40.9447716, -74.1260103]"
4,4,Salsa Fever On 2 Dance Academy,Dance Classes,,83 Franklin St,Jersey City,NJ,07307,"[40.7420044285714, -74.0483392857143]"


In [15]:
df = df.rename(columns={'index':'_id'})
df = df.drop(df.index[0])
#df = df.drop(columns=['Reviews'])
#df = df.drop(columns=['_id'])
df.head()

,_id,name,type,phoneNo,address,city,state,zipcode,location
1,1,The Institute of Music for Children,Dance Classes,,780 Salem Ave,Elizabeth,NJ,07208,"[40.6813924411765, -74.2172814705882]"
2,2,Just Dancin',Dance Classes,,187 Bloomfield Ave,Bloomfield,NJ,07003,"[40.6132314878049, -74.1833756829268]"
3,3,Dance Designs Studio,Dance Classes,,15-01 Pollitt Dr,Fair Lawn,NJ,07410,"[40.9447716, -74.1260103]"
4,4,Salsa Fever On 2 Dance Academy,Dance Classes,,83 Franklin St,Jersey City,NJ,07307,"[40.7420044285714, -74.0483392857143]"
5,5,Union County Dance Centre,Dance Classes,,911 W St Georges Ave,Linden,NJ,07036,"[40.6282193, -74.2717867]"


In [16]:
split_data = df.to_dict('records')
split_data

[{'_id': 1,
  'name': 'The Institute of Music for Children',
  'type': 'Dance Classes',
  'phoneNo': '',
  'address': ' 780 Salem Ave',
  'city': ' Elizabeth',
  'state': 'NJ',
  'zipcode': '07208',
  'location': [40.6813924411765, -74.2172814705882]},
 {'_id': 2,
  'name': "Just Dancin'",
  'type': 'Dance Classes',
  'phoneNo': '',
  'address': ' 187 Bloomfield Ave',
  'city': ' Bloomfield',
  'state': 'NJ',
  'zipcode': '07003',
  'location': [40.6132314878049, -74.1833756829268]},
 {'_id': 3,
  'name': 'Dance Designs Studio',
  'type': 'Dance Classes',
  'phoneNo': '',
  'address': ' 15-01 Pollitt Dr',
  'city': ' Fair Lawn',
  'state': 'NJ',
  'zipcode': '07410',
  'location': [40.9447716, -74.1260103]},
 {'_id': 4,
  'name': 'Salsa Fever On 2 Dance Academy',
  'type': 'Dance Classes',
  'phoneNo': '',
  'address': ' 83 Franklin St',
  'city': ' Jersey City',
  'state': 'NJ',
  'zipcode': '07307',
  'location': [40.7420044285714, -74.0483392857143]},
 {'_id': 5,
  'name': 'Union Co

In [17]:
df.dtypes

_id          int64
name        object
type        object
phoneNo     object
address     object
city        object
state       object
zipcode     object
location    object
dtype: object

In [19]:
# Set up the MongoDB connection through pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
# Create DB
db = myclient["KidsRidedb3"]
# Create collection and insert all the data into the MongoDB
mycol = db["activityData1"]
mycol.insert_many(split_data)